In [1]:
from __future__ import print_function
import sys, os
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator

import importlib
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /home/arno/h2o-3/build/h2o.jar
  Ice root: /tmp/tmpPZoblb
  JVM stdout: /tmp/tmpPZoblb/h2o_arno_started_from_python.out
  JVM stderr: /tmp/tmpPZoblb/h2o_arno_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,46 minutes
H2O cluster name:,H2O_from_python_arno_om38dl
H2O cluster total nodes:,1
H2O cluster free memory:,13.96 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [2]:
if not H2ODeepWaterEstimator.available(): quit()

# LeNET

Here we define the famous LENET neural network, but you can define any deep neural network of your choice.

In [3]:
def lenet(num_classes):
    import mxnet as mx
    data = mx.symbol.Variable('data')
    # first conv
    conv1 = mx.symbol.Convolution(data=data, kernel=(5,5), num_filter=20)
    tanh1 = mx.symbol.Activation(data=conv1, act_type="tanh")
    pool1 = mx.symbol.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
    # second conv
    conv2 = mx.symbol.Convolution(data=pool1, kernel=(5,5), num_filter=50)
    tanh2 = mx.symbol.Activation(data=conv2, act_type="tanh")
    pool2 = mx.symbol.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
    # first fullc
    flatten = mx.symbol.Flatten(data=pool2)
    fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
    tanh3 = mx.symbol.Activation(data=fc1, act_type="tanh")
    # second fullc
    fc2 = mx.symbol.FullyConnected(data=tanh3, num_hidden=num_classes)
    # loss
    lenet = mx.symbol.SoftmaxOutput(data=fc2, name='softmax')
    return lenet

## DeepWater for MXNET

We can use the import functions provided by h2o to import the list of 

In [4]:
train = h2o.import_file("../../bigdata/laptop/mnist/train.csv.gz")
test = h2o.import_file("../../bigdata/laptop/mnist/test.csv.gz")

predictors = list(range(0,784))
resp = 784
train[resp] = train[resp].asfactor()
test[resp] = test[resp].asfactor()
nclasses = train[resp].nlevels()[0]

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%


Let's create the lenet model architecture from scratch using the MXNet Python API

In [5]:
model = lenet(nclasses)

To import the model inside the DeepWater training engine we need to save the model to a file:

In [6]:
model_path = "/tmp/symbol_lenet-py.json"
model.save(model_path)

The model is just the structure of the network expressed as a json dict

In [7]:
!head "/tmp/symbol_lenet-py.json"

{
  "nodes": [
    {
      "op": "null", 
      "param": {}, 
      "name": "data", 
      "inputs": [], 
      "backward_source_id": -1
    }, 
    {


## Importing the LeNET model architecture for training in H2O

We have defined the model and saved the structure to a file. We are ready to start the training procedure.

In [8]:
model = H2ODeepWaterEstimator(epochs=100, learning_rate=1e-3, 
                              mini_batch_size=64,
                              network='user', 
                              network_definition_file=model_path,
                              image_shape=[28,28], channels=1)

In [9]:
model.train(x=predictors,y=resp, training_frame=train, validation_frame=test)

deepwater Model Build progress: |█████████████████████████████████████████████████████████████| 100%


In [10]:
model.show()

Model Details
H2ODeepWaterEstimator :  Deep Water
Model Key:  DeepWater_model_python_1475999056235_1
Status of Deep Learning Model: user: 1.6 MB, predicting C785, 10-class classification, 5,165,056 training samples, mini-batch size 64



,rate,momentum
,0.0001622,0.99




ModelMetricsMultinomial: deepwater
** Reported on train data. **

MSE: 0.00483862036448
RMSE: 0.0695601923839
LogLoss: 0.021180403921
Mean Per-Class Error: 0.00382060051535
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
133137663984000000.0000000,0.0,0.0,0.0,0.0,0.0,140737488355000.0000000,0.0,0.0,0.0,0.0010560,"140,737,488,355,328 / 133,278,401,472,495,616"
140737488355000.0000000,161285161655000000.0000000,140737488355000.0000000,0.0,0.0,0.0,0.0,281474976711000.0000000,140737488355000.0000000,0.0,0.0043440,"703,687,441,776,640 / 161,988,849,096,982,528"
140737488355000.0000000,0.0,141159700820000000.0000000,0.0,0.0,0.0,0.0,140737488355000.0000000,0.0,0.0,0.0019900,"281,474,976,710,656 / 141,441,175,797,104,640"
0.0,0.0,140737488355000.0000000,143411500634000000.0000000,0.0,0.0,0.0,281474976711000.0000000,140737488355000.0000000,281474976711000.0000000,0.0058537,"844,424,930,131,968 / 144,255,925,564,211,200"
0.0,0.0,0.0,0.0,135811676263000000.0000000,0.0,0.0,0.0,0.0,422212465066000.0000000,0.0030992,"422,212,465,065,984 / 136,233,888,727,957,504"
140737488355000.0000000,0.0,0.0,281474976711000.0000000,0.0,124552677194000000.0000000,0.0,0.0,0.0,0.0,0.0033784,"422,212,465,065,984 / 124,974,889,659,531,264"
140737488355000.0000000,0.0,0.0,0.0,0.0,0.0,140737488355000000.0000000,0.0,140737488355000.0000000,0.0,0.0019960,"281,474,976,710,656 / 141,018,963,332,038,656"
140737488355000.0000000,140737488355000.0000000,281474976711000.0000000,0.0,140737488355000.0000000,0.0,0.0,139048638495000000.0000000,0.0,0.0,0.0050352,"703,687,441,776,640 / 139,752,325,936,840,704"
0.0,140737488355000.0000000,0.0,140737488355000.0000000,0.0,0.0,0.0,0.0,147915100261000000.0000000,0.0,0.0018993,"281,474,976,710,656 / 148,196,575,238,160,384"
281474976711000.0000000,140737488355000.0000000,0.0,0.0,281474976711000.0000000,140737488355000.0000000,0.0,281474976711000.0000000,140737488355000.0000000,131308076636000000.0000000,0.0095541,"1,266,637,395,197,952 / 132,574,714,030,718,976"


Top-10 Hit Ratios: 


...

Top-10 Hit Ratios: 


k,hit_ratio
1,0.9853
2,0.9967
3,0.9993
4,0.9998
5,0.9999000
6,1.0
7,1.0
8,1.0
9,1.0
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2016-10-09 00:44:24,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2016-10-09 00:44:25,3.211 sec,6005 obs/sec,0.0341333,1,2048.0,0.6863949,1.2820884,0.3610387,0.6828730,1.2682603,0.3221381
,2016-10-09 00:44:40,17.923 sec,22445 obs/sec,5.3589333,157,321536.0,0.1261795,0.2385204,0.0252002,0.1658834,0.1000103,0.0299448
,2016-10-09 00:44:48,25.321 sec,23261 obs/sec,8.1237333,238,487424.0,0.1353207,0.0699119,0.0194506,0.1511284,0.0828505,0.0235852
,2016-10-09 00:44:56,32.962 sec,25293 obs/sec,11.776,345,706560.0,0.1192561,0.0553768,0.0141368,0.1402123,0.0710302,0.0209124
---,---,---,---,---,---,---,---,---,---,---,---,---
,2016-10-09 00:47:14,2 min 51.614 sec,30209 obs/sec,76.9365333,2254,4616192.0,0.0620064,0.0176156,0.0030078,0.1126842,0.0448258,0.0138507
,2016-10-09 00:47:21,2 min 58.437 sec,30227 obs/sec,80.0426667,2345,4802560.0,0.0607476,0.0169945,0.0029076,0.1124622,0.0447294,0.0137238
,2016-10-09 00:47:28,3 min 5.914 sec,30213 obs/sec,83.4218667,2444,5005312.0,0.0601931,0.0166959,0.0029076,0.1122255,0.0889484,0.0138
,2016-10-09 00:47:35,3 min 12.937 sec,30036 obs/sec,86.0842667,2522,5165056.0,0.0590891,0.0162206,0.0028073,0.1119555,0.0443369,0.0137



See the whole table with table.as_data_frame()
